In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Descrição

    Classificação do tipo de vidro de acordo com o Índice de Refração 'RI' e a porcentagem de peso de 8 componentes químicos: 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'.

## Classes
1. building_windows_float_processed 
2. building_windows_non_float_processed 
3. vehicle_windows_float_processed
4. vehicle_windows_non_float_processed (**Nenhum neste dataset**)
5. containers 
6. tableware
7. headlamps

## Importando os dados

In [ ]:
data = pd.read_csv("../input/glass/glass.csv")
variables = data.iloc[:, :-1]
target = data.iloc[:, -1]

In [ ]:
for i in range(len(data['Type'])):
    if data['Type'][i] == 1:
        data['Type'][i] = 'building_windows_float_processed'
    elif data['Type'][i] == 2:
        data['Type'][i] = 'building_windows_non_float_processed'
    elif data['Type'][i] == 3:
        data['Type'][i] = 'vehicle_windows_float_processed'
    elif data['Type'][i] == 5:
        data['Type'][i] = 'containers'
    elif data['Type'][i] == 6:
        data['Type'][i] = 'tableware'
    elif data['Type'][i] == 7:
        data['Type'][i] = 'headlamps'
        
print(data['Type'])

## Número de variáveis e registros

In [ ]:
n_variables = len(variables.columns)
print('Número de variáveis = ', n_variables)
n_registers = len(variables['RI'])
print('Número de registros = ', n_registers)

## Estatísticas Básicas

In [ ]:
print('--- Mínimos ---')
print(np.amin(variables), '\n')
print('--- Máximos ---')
print(np.amax(variables), '\n')
print('--- Médias ---')
print(np.mean(variables), '\n')
print('--- Desvios Padrão ---')
print(np.std(variables), '\n')
print('--- Variâncias ---')
print(np.var(variables), '\n')

## Valores ausentes
    Segundo a fonte, não há valores ausentes, registros com valores 0 indicam a ausencia do elemento na composição do vidro.

## Box-Plots e Outliers

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(12, 8))

axs[0][0].boxplot(variables['RI'], widths = 0.7)
axs[0][0].set_title('RI')

axs[0][1].boxplot(variables['Na'], widths = 0.7)
axs[0][1].set_title('Na')

axs[0][2].boxplot(variables['Mg'], widths = 0.7)
axs[0][2].set_title('Mg')

axs[1][0].boxplot(variables['Al'], widths = 0.7)
axs[1][0].set_title('Al')

axs[1][1].boxplot(variables['Si'], widths = 0.7)
axs[1][1].set_title('Si')

axs[1][2].boxplot(variables['K'], widths = 0.7)
axs[1][2].set_title('K')

axs[2][0].boxplot(variables['Ca'], widths = 0.7)
axs[2][0].set_title('Ca')

axs[2][1].boxplot(variables['Ba'], widths = 0.7)
axs[2][1].set_title('Ba')

axs[2][2].boxplot(variables['Fe'], widths = 0.7)
axs[2][2].set_title('Fe')

fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.4, wspace=0.3)

In [ ]:
Q1 = variables.quantile(0.25)
Q3 = variables.quantile(0.75)
IQR = Q3 - Q1

print('\n--- Número de Outliers ---')
print(((variables < (Q1 - 1.5 * IQR)) | (variables > (Q3 + 1.5 * IQR))).sum())

Podemos ver um grande destaque para **Ba** em número de outliers (38), seguido de **Ca** (26). Ao mesmo tempo vemos que **Mg** não possui outliers.

## Histogramas

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(12, 8))

axs[0][0].hist(variables['RI'])
axs[0][0].set_title('RI')

axs[0][1].hist(variables['Na'])
axs[0][1].set_title('Na')

axs[0][2].hist(variables['Mg'])
axs[0][2].set_title('Mg')

axs[1][0].hist(variables['Al'])
axs[1][0].set_title('Al')

axs[1][1].hist(variables['Si'])
axs[1][1].set_title('Si')

axs[1][2].hist(variables['K'])
axs[1][2].set_title('K')

axs[2][0].hist(variables['Ca'])
axs[2][0].set_title('Ca')

axs[2][1].hist(variables['Ba'])
axs[2][1].set_title('Ba')

axs[2][2].hist(variables['Fe'])
axs[2][2].set_title('Fe')

fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.4, wspace=0.3)

Observamos distribuições semelhantes à normal em **RI**, **Na**, **Al**, **Si** e **Ca**. A grande maioria de valores de **Ba**, **Fe** e **K** próximos de 0.0. E, em **Mg**, a presença de valores tanto próximos de 0 quanto entre 3.0 e 4.0, o que já tinha sido indicado pela sua ausência de outliers.

## Matriz de correlação
* 0- RI
* 1- Na
* 2- Mg
* 3- Al
* 4- Si
* 5- K
* 6- Ca
* 7- Ba
* 8- Fe

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.matshow(variables.corr())
plt.title('Matriz de Correlação')
cb = plt.colorbar()
cb.ax.tick_params(labelsize=10)
plt.show()

## Gráfico de Projeção

In [ ]:
sns.pairplot(data, hue="Type", diag_kind="hist")

Podemos visualizar a alta correlação do par de variáveis **RI** e **Ca**, o que é confirmado pela matriz de correlação.